# 数据预处理

## 加载数据与基本数据处理

In [1]:
import pandas as pd
import datetime
import numpy as np

In [2]:
orders= pd.read_csv('data/orders.csv',index_col=0)
orders

In [3]:
orders.info()

- 这里为了演示RFM的划分, 缺失数据直接删除

In [4]:
orders.dropna(inplace = True)

- 替换数据中的英文

In [5]:
orders['product'] = orders['product'].replace('banana', '香蕉')
# 另外一种替换方式
orders['product'].replace('milk', '牛奶',inplace = True)
orders['product'].replace('water', '水',inplace = True)

## RFM计算

In [179]:
orders

In [6]:
orders['values'] = 1
purchase_list = orders.pivot_table(index=['clientId','orderId','gender','orderdate'], #行索引
                          columns='product', # 列索引
                          aggfunc=sum, # 计算方式，max, min, mean, sum, len
                          values='values' #值
                          ).fillna(0).reset_index()
purchase_list

- 频率计算

In [7]:
# 创建一列 frequency 辅助计算
purchase_list['frequency'] = 1
frequency = purchase_list.groupby("clientId", #按照用户ID进行分组
                                  as_index = False # 分类的列是否做为行索引
                                  )['frequency'].sum() # 聚合方式，max, min, mean, sum
frequency

In [8]:
# 完成计算后可以将辅助计算列删除
del purchase_list['frequency']
purchase_list

- 最近一次消费计算

In [9]:
# 将数据中最后一天作为观察日期, 以这一天为基准分析RFM情况
theToday = datetime.datetime.strptime(orders['orderdate'].max(), "%Y-%m-%d")
# 将数据中的'orderdate'列处理成日期时间格式
purchase_list['orderdate'] = pd.to_datetime(purchase_list['orderdate'])
# 计算每个顾客最近一次购买日期
recent_recency = purchase_list.groupby("clientId", as_index = False)['orderdate'].max()
# 计算 观察日 与 每位用户最后一次购买日期的时间差
recent_recency['recency'] =( theToday - recent_recency['orderdate'] ).astype(str)
recent_recency

In [10]:
# 去掉recency列中的days
recent_recency['recency'] = recent_recency['recency'].str.replace('days.*', #要替换的内容
                                                                  '', #替换成的字符串
                                                                  regex = True)
# 'recency'列转换成Int
recent_recency['recency'] = recent_recency['recency'].astype(int)
recent_recency

- 合并recency、frequency

In [11]:
purchase_list = recent_recency.merge(purchase_list, # 要合并的DataFrame
                                     on = ['clientId', 'orderdate'] # 链接的Key
                                     ,how='inner') # 合并的方式

In [12]:
purchase_list =purchase_list.merge(frequency, # 要合并的DataFrame
                                   on = ['clientId'] # 链接的Key
                                   ,how='inner') # 合并的方式
purchase_list

- 划分recency、frequency

In [13]:
# 将recency按照时间远近分组
recency_label =  ['0-7 day', '8-15 day', '16-22 day', '23-30 day', '31-55 day', '>55 day']
# cut 自定义的方式对数据进行分组, 默认 左开右闭
recency_cut  = [-1, 7, 15, 22, 30, 55, purchase_list['recency'].max()]
purchase_list['recency_cate'] = pd.cut( 
        purchase_list['recency'] , #要分组的列
        recency_cut, #划分条件
        labels =recency_label) #每组的名字

# 将frequency按照频率高低分组
frequency_label =  ['1 freq', '2 freq', '3 freq', '4 freq', '5 freq', '>5 freq']
frequency_cut  = [0, 1, 2, 3, 4, 5, purchase_list['frequency'].max()]
purchase_list['frequency_cate'] = pd.cut( 
        purchase_list['frequency'] , #要分组的列
        frequency_cut,  #划分条件
        labels =frequency_label) #每组的名字
purchase_list

- RFM分析

In [171]:
str.join?

In [14]:
# RF交叉分析
RF_table = pd.crosstab(purchase_list['frequency_cate'].astype(str),
                       purchase_list['recency_cate'].astype(str))

# 重新排序
RF_table['freq'] = RF_table.index
RF_table = RF_table.sort_values('freq',ascending = False)

collist = ['freq'] + recency_label
RF_table = RF_table[collist]

In [15]:
RF_table

In [16]:
# 根据RF标注出顾客类别 常客,新客,沉睡客,流失客
purchase_list['customer'] = np.where( (purchase_list['frequency'] >=frequency_cut[4]) & (purchase_list['recency']<=recency_cut[3]), '常客',
                     np.where( (purchase_list['frequency'] >=frequency_cut[4]) & ( purchase_list['recency']>recency_cut[3]), '沉睡客',
                              np.where( (purchase_list['frequency'] < frequency_cut[4]) & ( purchase_list['recency']>recency_cut[3]), '流失客',
                                       '新客'  )))

purchase_list.to_csv('purchase_list.csv')
purchase_list

# RFM可视化

## Seaborn绘图

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
rc = {'font.sans-serif': 'Arial Unicode MS', # Mac系统
# rc = {'font.sans-serif': 'SimHei', # windows系统      
      'axes.unicode_minus': False,
     'figure.figsize':(10,8)}
sns.set(context='notebook', style='ticks', rc=rc)

In [18]:
#准备XY轴标签
recency_label =  ['0-7 day', '8-15 day', '16-22 day', '23-30 day', '31-55 day', '>55 day']
frequency_label =  ['1 freq', '2 freq', '3 freq', '4 freq', '5 freq', '>5 freq']

#绘图
g = sns.FacetGrid(purchase_list, # 数据源
                  col="recency_cate", # X轴数据来源
                  row="frequency_cate" ,  # Y轴数据来源
                  col_order= recency_label,  # X方向数据顺序
                  row_order= frequency_label, # Y方向数据顺序
                  palette='Set1',  #画布色调
                  margin_titles=True)
#小图表部分
g = g.map_dataframe(sns.barplot, y ='水')
g = g.set_axis_labels('距今','频率').add_legend()

## matplotlib绘制RFM

In [19]:
#准备XY轴标签
recency_label =  ['0-7 day', '8-15 day', '16-22 day', '23-30 day', '31-55 day', '>55 day']
frequency_label =  ['1 freq', '2 freq', '3 freq', '4 freq', '5 freq', '>5 freq']


#绘图
#先设定画布大小   6*6 代表有36张小图
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 画布X轴坐标
for i in frequency_label[::-1]: # 我们内层的小图是从上向下绘制的, 所以要先绘制频率>5次的组别, 需要将列表倒转遍历
    countY = 0 # 画布Y轴坐标
    for j in recency_label: # 最近几天内来过
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data.shape[0] != 0: # 检查这个位置有没有数据
            # 以下为单一小图表的设定
            sns.barplot(y="牛奶", # 小图表Y数据
                        data=data, #数据来源
                        estimator=np.sum,
                        capsize=.2, # 最高点最低点大小
                        ax=axes[countX, countY]) # 小图表坐标

        countY += 1 
    countX += 1 
fig.suptitle('RFM图', position=(.5,1), fontsize=35) # 标题
fig.text(0.5, 0.01, '光顾天数', ha='center', va='center', fontsize=20) # 设定X轴标题
fig.text(0.01, 0.5, '购买频率', ha='center', va='center', rotation='vertical', fontsize=20) # 设定Y轴标题
fig.show()

# RFM分析-产品分析

## RFM图调整 XY轴统一标签

In [20]:
#先设定画布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 画布X轴坐标
for i in frequency_label[::-1]: # 我们内层的小图是从上向下绘制的, 所以要先绘制频率>5次的组别, 需要将列表倒转遍历
    countY = 0 # 画布Y轴坐标
    for j in recency_label: # 近因
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data.shape[0] != 0: #检查这部分有没有数据
            # 下面设定单一小图表
            sns.barplot(y="牛奶", # 小图标Y数据来源
                        data=data, #绘图使用数据
                        estimator=np.sum,
                        capsize=.2, # 最高点最低点大小
                        ax=axes[countX, countY]) # 小图表坐标
        ################ 画X标记################
        if i == '1 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ############### 画Y标记 ################
        if j == '0-7 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM图', position=(.5,1), fontsize=35) # 标题
fig.text(0.5, 0.01, '光顾天数', ha='center', va='center', fontsize=20) # X轴标题
fig.text(0.01, 0.5, '购买频率', ha='center', va='center', rotation='vertical', fontsize=20) # Y轴标题
fig.show()

## 统一Y轴刻度

In [21]:
findbig=0
for i in frequency_label:
    for j in recency_label:
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data['牛奶'].sum() > findbig:
            findbig = data['牛奶'].sum()


#先设定画布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 画布X轴坐标
for i in frequency_label[::-1]: # 由于axes画布排列的关系，频率必须要反着放
    countY = 0 # 画布Y轴坐标
    for j in recency_label: # 近因
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data.shape[0] != 0: #检查这部分有没有数据
            # 下面设定单一小图表
            sns.barplot(y="牛奶", # 小图标Y数据来源
                        data=data, #绘图使用数据
                        estimator=np.sum,
                        capsize=.2, # 最高点最低点大小
                        ax=axes[countX, countY]) # 小图表坐标

        ################ 将水牛奶香蕉的字体变大 ################
        axes[countX][countY].tick_params(labelsize=15)
        ############### 使所有数据尺码相同 ################
        axes[countX][countY].set_yticks(range(0,int(findbig*1.3),10))
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM图', position=(.5,1), fontsize=35) # 标题
fig.text(0.5, 0.01, '光顾天数', ha='center', va='center', fontsize=20) # X轴标题
fig.text(0.01, 0.5, '购买频率', ha='center', va='center', rotation='vertical', fontsize=20) # Y轴标题
fig.show()



## 四大类顾客分群

In [22]:
findbig=0
for i in frequency_label:
    for j in recency_label:
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data['牛奶'].sum() > findbig:
            findbig = data['牛奶'].sum()


#先设定画布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 画布X轴坐标
for i in frequency_label[::-1]: # 由于axes画布排列的关系，频率必须要反着放
    countY = 0 # 画布Y轴坐标
    for j in recency_label: # 近因
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data.shape[0] != 0: #检查这部分有没有数据
            # 下面设定单一小图表
            sns.barplot(y="牛奶", # 小图标Y数据来源
                        data=data, #绘图使用数据
                        estimator=np.sum,
                        capsize=.2, # 最高点最低点大小
                        ax=axes[countX, countY]) # 小图表坐标
############### 四个区块分颜色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #紅色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黄色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #蓝色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #绿色
            
        countY += 1 
    countX += 1
    
fig.suptitle('RFM图', position=(.5,1), fontsize=35) # 标题
fig.text(0.5, 0.01, '光顾天数', ha='center', va='center', fontsize=20) # X轴标题
fig.text(0.01, 0.5, '购买频率', ha='center', va='center', rotation='vertical', fontsize=20) # Y轴标题
fig.show()

In [23]:
sns.barplot(y="牛奶", # 小图标Y数据来源
            data=data, #绘图使用数据
            estimator=np.sum,
            capsize=.2, # 最高点最低点大小
            ) # 小图表坐标

## RFM产品分析

In [24]:
purchase_list.drop(columns = ['orderId','orderdate','recency','frequency'])

In [25]:
temp = purchase_list.drop(columns = ['orderId','orderdate','recency','frequency'])
# 将部分数据 宽变长, id_vars 保留不去处理的字段   
df3 = pd.melt(temp, id_vars=['clientId','customer','recency_cate','frequency_cate','gender'], var_name='types', value_name='values') 
df3

In [26]:
df3 = pd.melt(purchase_list.drop(columns = ['orderId','orderdate','recency','frequency']), id_vars=['clientId','customer','recency_cate','frequency_cate','gender'], var_name='types', value_name='values') 
df3['values'] = pd.to_numeric(df3['values'],errors='coerce')
df3 = df3.dropna()

#先设定画布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 画布X轴坐标
for i in frequency_label[::-1]: # 我们内层的小图是从上向下绘制的, 所以要先绘制频率>5次的组别, 需要将列表倒转遍历
    countY = 0 # 画布Y轴坐标
    for j in recency_label: # 最近R标签数据
        data = df3[(df3['recency_cate']==j) & (df3['frequency_cate']==i)]
        if data.shape[0] != 0: #检查这部分有没有数据
            # 下面绘制每一个小图表
            sns.barplot(x="types", # 小图表X轴数据
                        y="values", # 小图表Y轴数据
                        data=data, #绘图用到的数据
                        capsize=.2, 
                        ax=axes[countX, countY]) # 小图表坐标
        # ################ 画X轴刻度 ################
        if i == '1 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ############### 画Y轴刻度 ################
        if j == '0-7 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
            
        ################ 将水、牛奶、香蕉的字变大################
        axes[countX][countY].tick_params(labelsize=15)
        ############### 统一所有小图的Y轴刻度大小 ################
        axes[countX][countY].set_yticks(range(0,10,3))
        
        
        ###############四个区块分颜色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #红色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黃色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #蓝色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #绿色
            
        countY += 1 
    countX += 1 

fig.suptitle('RFM产品分析图', position=(.5,1), fontsize=35) # 标题
fig.text(0.5, 0.01, '光顾天数', ha='center', va='center', fontsize=20) # X轴标题
fig.text(0.01, 0.5, '购买频率', ha='center', va='center', rotation='vertical', fontsize=20) # Y轴标题
fig.show()

# RFM分析-市场分析

## 市场分析

In [27]:
df3 = pd.melt(purchase_list.drop(columns = ['orderId','orderdate','recency','frequency']), id_vars=['clientId','customer','recency_cate','frequency_cate','gender'], var_name='types', value_name='values') 
df3['values'] = pd.to_numeric(df3['values'],errors='coerce')
df3 = df3.dropna()


In [28]:
df3

In [47]:
data = df3[(df3['recency_cate']=='8-15 day') & (df3['frequency_cate']=='5 freq')]

In [48]:
data

In [49]:
data = data[['gender','types','values']].groupby(['types','gender']).sum()


In [ ]:
data1 
A B
1 2 
3 4

In [ ]:
data.columns = [['C', 'D']]

In [50]:
data.index

In [36]:
data

In [37]:
data = data.groupby(level=1).apply(lambda x:100 * x / float(x.sum()))

In [38]:
data

In [39]:
data = data.add_suffix('').reset_index() #multiIndex 变平

In [40]:
data

In [41]:
data= data.pivot('gender', 'types', 'values')

In [42]:
data

In [43]:
data.plot.bar(stacked=True, # 设置堆积图
              width=0.7,# 柱子的宽度
              legend = True, 
            # 小图表坐标
              rot=0) #坐标轴文字旋转

In [44]:

#先设定画布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 画布X轴坐标
for i in frequency_label[::-1]: # 我们内层的小图是从上向下绘制的, 所以要先绘制频率>5次的组别, 需要将列表倒转遍历
    countY = 0 # 画布Y轴坐标
    for j in recency_label: # 近因
        data = df3[(df3['recency_cate']==j) & (df3['frequency_cate']==i)]
        if data.shape[0] != 0: # 检查这部分有没有数据
            #处理堆叠数据，将这部分数据的购买量,换算成百分比data = data.groupby(['types', 'gender'])['values'].sum() # 依照不同的性別，对购买量求和
            data = data[['gender','types','values']].groupby(['types','gender']).sum()
            data = data.groupby(level=1).apply(lambda x:100 * x / float(x.sum())) # 换算成百分比
            data = data.add_suffix('').reset_index() #multiIndex 变平
            data = data.pivot('gender', 'types', 'values') # 透视表
            
           # 下面设定单一小图表
            ax = data.plot.bar(stacked=True, # 设置堆积图
                              width=0.7,# 柱子的宽度
                              legend = True, 
                              ax =axes[countX, countY] , # 小图表坐标
                              rot=0) #坐标轴文字旋转
            
        ################ 画X标记 ################
        if i == '1 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ############### 画Y标记 ################
        if j == '0-7 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
            
        ################ 将水、牛奶、香蕉的字变大 ################
        axes[countX][countY].tick_params(labelsize=15)
        
        
        ############### 四个区块分颜色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") 
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") 
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") 
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") 
            
        countY += 1 
    countX += 1
fig.suptitle('RFM-市场分析图', position=(.5,1), fontsize=35) # 标题
fig.text(0.5, 0.01, '光顾天数', ha='center', va='center', fontsize=20) # X轴标题
fig.text(0.01, 0.5, '购买频率', ha='center', va='center', rotation='vertical', fontsize=20) # Y轴标题
fig.show()

In [45]:
data.index

In [35]:
recency_label =  ['0-7 day', '8-15 day', '16-22 day', '23-30 day', '31-55 day', '>55 day']
frequency_label =  ['1 freq', '2 freq', '3 freq', '4 freq', '5 freq', '>5 freq']

In [36]:
data = df3[(df3['recency_cate']=='0-7 day') & (df3['frequency_cate']== '>5 freq')]

In [37]:
data = data[['gender','types','values']].groupby(['types','gender']).sum()
data

In [38]:
data.index.levels[1]

- 此时可以通过level操作MultiIndex的数据, 也可以将其变为普通索引

In [39]:
# 参数level 如果是MultiIndex 可以按照level 的索引进行分组
data =data.groupby(level=1).apply(lambda x:100 * x / float(x.sum())) # 将具体值换成百分比表示
data

In [40]:
data = data.add_suffix('').reset_index() #将MultiIndex转变成普通索引
data


- 创建透视表，完成数据准备

In [41]:
data=data.pivot('gender', 'types', 'values') # 透视
data

In [42]:
ax = data.plot.bar(stacked=True, # 设定堆积图
                              width=0.7,# 柱子的宽度
                              legend = True, 
                              rot=0) 

## 市场分析图例优化

In [43]:
#先设定画布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 画布X轴坐标
for i in frequency_label[::-1]: # 我们内层的小图是从上向下绘制的, 所以要先绘制频率>5次的组别, 需要将列表倒转遍历
    countY = 0 # 画布Y轴坐标
    for j in recency_label: # 近因
        data = df3[(df3['recency_cate']==j) & (df3['frequency_cate']==i)]
        if data.shape[0] != 0: #检查这部分有没有数据
            # 处理堆叠数据，将这部分数据的购买量,换算成百分比data = data.groupby(['types', 'gender'])['values'].sum() # 依照不同的性別，对购买量求和
            data = data[['gender','types','values']].groupby(['types','gender']).sum()
            data =data.groupby(level=1).apply(lambda x:100 * x / float(x.sum())) # 换算成百分比
            data = data.add_suffix('').reset_index() #multiIndex 变平
            data=data.pivot('gender', 'types', 'values') # 透视表
            
             # 下面设定单一小图表
            ax = data.plot.bar(stacked=True, # 设置堆积图
                              width=0.7,# 柱子的宽度
                              legend = False, 
                              ax =axes[countX, countY] , # 小图表坐标
                              rot=0) #坐标轴文字旋转
            
        ################ 设定图例 ################
        if (i == '4 freq') and (j == '>55 day'):
            ax.legend(bbox_to_anchor=(1.03, 0.8), loc=2, fontsize =20) #设定图例
            
        ################ 画X标记 ################
        if i == '1 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ###############  画Y标记 ################
        if j == '0-7 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
            
        ################ 将水、牛奶、香蕉的字变大 ################
        axes[countX][countY].tick_params(labelsize=15)
        
        
        ############### 四个区块分颜色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #红色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黃色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #蓝色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #绿色
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM-市场分析图', position=(.5,1), fontsize=35) # 标题
fig.text(0.5, 0.01, '光顾天数', ha='center', va='center', fontsize=20) # X轴标题
fig.text(0.01, 0.5, '购买频率', ha='center', va='center', rotation='vertical', fontsize=20) # Y轴标题
fig.show()



# RFM分析 - 财务分析

## 成本获利分析

In [51]:
cac = pd.read_csv('data/cac.csv', index_col=0)
cac

In [52]:
orders

In [53]:
# 计算clvs
clv = orders[['clientId','grossmarg']].groupby('clientId').sum().reset_index() #计算每个顾客的总消费金额
clv

In [58]:
clv.columns = ['clientId', 'clv'] #重命名列名
clv

In [59]:
# purchase_list,clv,cac 合起来（merge）
purchase_list =  purchase_list.merge(clv,on=['clientId'])
purchase_list =  purchase_list.merge(cac,on=['clientId'])
purchase_list

In [61]:
purchase_list[['frequency_cate', 'recency_cate','clv', 'cac']].groupby(['frequency_cate','recency_cate']).sum().reset_index()

In [63]:
# 计算不同分组的clv与cac
countfinal = purchase_list[['frequency_cate', 'recency_cate','clv', 'cac']].groupby(['frequency_cate', 'recency_cate']).sum().reset_index()
countfinal

In [65]:
pd.melt(countfinal, id_vars=['frequency_cate','recency_cate'], value_vars=['clv', 'cac'])

In [67]:

# 将clv与cac做melt转换
finaldf = pd.melt(countfinal, id_vars=['frequency_cate','recency_cate'], value_vars=['clv', 'cac'])
finaldf

In [68]:
# 缺失值补0
finaldf['value'] = finaldf['value'].fillna(0)
finaldf.to_csv('finaldf.csv' , index=0)

In [69]:
finaldf

In [70]:
#先设定画布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 #  画布X轴坐标
for i in frequency_label[::-1]: # 我们内层的小图是从上向下绘制的, 所以要先绘制频率>5次的组别, 需要将列表倒转遍历
    countY = 0 # 画布Y轴坐标
    for j in recency_label: # 近因
        data = finaldf[(finaldf['recency_cate']==j) & (finaldf['frequency_cate']==i)]
        if data.shape[0] != 0: #  检查这部分有没有数据
            # 下面设定单一小图表
            sns.barplot(x="variable", # 指定小图标X轴数据
                        y="value", # 指定小图标y轴数据
                        data=data, #绘图使用的DataFrame
                        capsize=.2, 
                        ax=axes[countX, countY]) # 小图表坐标
            
            
        ################ 设定图例 ################
        # if (i == '4 freq') and (j == '>55 day'):
        #     axes.legend(bbox_to_anchor=(1.03, 0.8), loc=2, fontsize =20) #设定图例  
            
        ################ 画X标签 ################
        if i == '1 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ############### 画Y标签 ################
        if j == '0-7 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
            
        ################ 将水、牛奶、香蕉的字变大 ################
        axes[countX][countY].tick_params(labelsize=15)
        
        
        ############### 四个区块分颜色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #红色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黄色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #蓝色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #绿色
            
        countY += 1 
    countX += 1 

fig.suptitle('RFM-成本获利分析图', position=(.5,1), fontsize=35) # 标题
fig.text(0.5, 0.01, '光顾天数', ha='center', va='center', fontsize=20) # X轴标题
fig.text(0.01, 0.5, '购买频率', ha='center', va='center', rotation='vertical', fontsize=20) # Y轴标题
fig.show()

## RFM毛利率分析

In [71]:
countfinal['毛利'] = countfinal['clv'] - countfinal['cac']
countfinal['毛利检查'] = np.where(countfinal['毛利']< 0,'#ff5858','#58acff')

In [72]:
#先设定画布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 画布X轴坐标
for i in frequency_label[::-1]: # 我们内层的小图是从上向下绘制的, 所以要先绘制频率>5次的组别, 需要将列表倒转遍历
    countY = 0 #画布Y轴坐标
    for j in recency_label: # 近因
        data = countfinal[(countfinal['recency_cate']==j) & (countfinal['frequency_cate']==i)]
        if data.shape[0] != 0: # 检查这部分有没有数据
            # 下面设定单一小图表
            sns.barplot( #  指定小图标X轴数据
                        y="毛利", #  指定小图标Y轴数据
                        data=data, #绘图使用的DataFrame
                        capsize=.2, 
                        color=data['毛利检查'].values[0],
                        ax=axes[countX, countY]) # 小图表坐标
            
              
        ################ 画X标签 ################
        if i == '1 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ###############画Y标签 ################
        if j == '0-7 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
            
        ############### 使所有数据的大小相同 ################
        axes[countX][countY].set_yticks(range(-200,12000,3000))
        
        ############### 四个区块分颜色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #红色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黄色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #蓝色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #绿色
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM-毛利分析图', position=(.5,1), fontsize=35) # 标题
fig.text(0.5, 0.01, '光顾天数', ha='center', va='center', fontsize=20) # X轴标题
fig.text(0.01, 0.5, '购买频率', ha='center', va='center', rotation='vertical', fontsize=20) # Y轴标题
fig.show()

## RFM 投资回报率分析

In [73]:
countfinal['ratio'] = countfinal['clv'] / countfinal['cac']
countfinal['ratio'] = countfinal['ratio'].round(2)
countfinal['ratio'] = countfinal['ratio'].fillna(0)
countfinal['ratio_index'] = np.where(countfinal['ratio']< 3,'#ff5858','#58acff')
countfinal

In [74]:
#先设定画布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 #画布X轴坐标
for i in frequency_label[::-1]: # 我们内层的小图是从上向下绘制的, 所以要先绘制频率>5次的组别, 需要将列表倒转遍历
    countY = 0 # 画布Y轴坐标
    for j in recency_label: # 近因
        data = countfinal[(countfinal['recency_cate']==j) & (countfinal['frequency_cate']==i)]
        if data.shape[0] != 0: # 检查这部分有没有数据
            # 下面设定单一小图表
            sns.barplot( # 指定小图表X轴数据
                        y="ratio", #指定小图标Y轴数据
                        data=data, #数据
                        capsize=.2, 
                        color=data['ratio_index'].values[0],
                        ax=axes[countX, countY]) # 小图表坐标
            
              
         ################ 画X标签 ################
        if i == '1 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
         ################ 画y标签 ################
        if j == '0-7 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
        ############### 使所有数据的大小相同 ################
        axes[countX][countY].set_yticks(range(0,10,2))
        
        ############### 四个区块分颜色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #红色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黄色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #蓝色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #绿色     
        countY += 1 
    countX += 1 
fig.suptitle('RFM-投资回报率分析图', position=(.5,1), fontsize=35) # 标题
fig.text(0.5, 0.01, '光顾天数', ha='center', va='center', fontsize=20) # X轴标题
fig.text(0.01, 0.5, '购买频率', ha='center', va='center', rotation='vertical', fontsize=20) # Y轴标题
fig.show()

# RFM营销分析

- 这里假设不同客群的ROI在试算过程中是固定的

In [172]:
countfinal['cac'] 

In [178]:
(countfinal[countfinal['ratio'] < 3]['cac']/2).mean()

In [ ]:
((countfinal[countfinal['ratio'] < 3]['cac']/2).mean())/2 

In [75]:
#营销数据计算
total_cac = countfinal['cac'].sum() #总成本
total_clv = countfinal['clv'].sum() #总收益

#将所有转化率低的成本移到高的部分
countfinal['cac'] = np.where(countfinal['ratio'] < 3 , 
                             countfinal['cac']/2, 
                             countfinal['cac'] + ((countfinal[countfinal['ratio'] < 3]['cac']/2).mean())/2 )

countfinal['clv'] = np.where(countfinal['ratio'] < 3 ,
                             countfinal['clv']/2, 
                             countfinal['ratio'] * countfinal['cac'])

countfinal['ratio'] = countfinal['clv'] / countfinal['cac']

- 计算新的毛利

In [76]:
countfinal['新毛利'] = countfinal['clv'] - countfinal['cac']
countfinal['新毛利检查'] = np.where(countfinal['新毛利']< 0,'#ffa6a6','#a6ffff')

In [77]:
#先设定画布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 画布X轴坐标
for i in frequency_label[::-1]: # 我们内层的小图是从上向下绘制的, 所以要先绘制频率>5次的组别, 需要将列表倒转遍历
    countY = 0 # 画布Y轴坐标
    for j in recency_label: # 近因
        data = countfinal[(countfinal['recency_cate']==j) & (countfinal['frequency_cate']==i)]
        if data.shape[0] != 0: #  检查这部分有没有数据
            #下面设定单一小图表
            sns.barplot( # 指定小图表X据
                        y="新毛利", # 指定小图标表Y轴数据
                        data=data, #绘图使用的DataFrame
                        capsize=.2, 
                        color=data['新毛利检查'].values[0],
                        ax=axes[countX, countY]) # 小图表坐标
            
            sns.barplot( # 指定小图表X据
                        y="毛利", # 指定小图标表Y轴数据
                        data=data, #绘图使用的DataFrame
                        capsize=.2, 
                        color=data['毛利检查'].values[0],
                        ax=axes[countX, countY]) # 小图表坐标
         ################ 画X标签 ################
        if i == '1 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ############### 画y标签 ################
        if j == '0-7 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
            
        ################ 将水、牛奶、香蕉的字变大 ################
        axes[countX][countY].tick_params(labelsize=15)
        ############### 使所有数据的大小相同  ################
        axes[countX][countY].set_yticks(range(-200,12000,3000))
        
        ############### 四个区块分颜色  ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #红色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黄色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #蓝色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #绿色
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM-毛利分析图', position=(.5,1), fontsize=35) # 标题
fig.text(0.5, 0.01, '光顾天数', ha='center', va='center', fontsize=20) # X轴标题
fig.text(0.01, 0.5, '购买频率', ha='center', va='center', rotation='vertical', fontsize=20) # Y轴标题
fig.show()

In [78]:
import numpy as np

In [79]:
np.where?

# RFM顾客复购分析

## 复购分析

In [107]:
orders2 = pd.read_csv('data/orders_2.csv', index_col=0)
orders2.dropna(inplace = True)

In [108]:
orders2

In [131]:
orders2['values'] = 1
purchase_list2 = orders2.pivot_table(index=['clientId','gender','orderdate'], #分类条件
                          columns='product', # 透视表列名
                          aggfunc=sum, # 计算方式，max, min, mean, sum, len
                          values='values' #值
                          ).fillna(0).reset_index()

##### 频率计算 #####
#计算每个消费者在一定时期内购买产品的次数
purchase_list2['frequency'] = 1
frequency = purchase_list2.groupby("clientId", #分类条件
                                  as_index = False # 分类条件是否要取代Index
                                  )['frequency'].sum() 
# 删除该字段
del purchase_list2['frequency']

# 合并 frequency 
purchase_list2 =purchase_list2.merge(frequency, #即将合并的数据
                                   on = ['clientId'] # 两者时作为
                                   ,how='inner') # 合并的方式

##### 最近一次消费计算 #####
# 假设今天的日期就是数据集中最大的购买日期, 我们以这一天为基准考复购情况
theToday = datetime.datetime.strptime(orders['orderdate'].max(), "%Y-%m-%d")
# 转换日期时间格式
purchase_list2['orderdate'] = pd.to_datetime(purchase_list2['orderdate'])
# 计算theToday距离最后一次消费的时间差
purchase_list2['recency'] =( theToday - purchase_list2['orderdate'] ).astype(str)
# 將'recency'列中的days后缀去掉
purchase_list2['recency'] = purchase_list2['recency'].str.replace('days.*', #想取代的東西
                                                                  '', #取代成的東西
                                                                  regex = True)
# 将'recency'列全部转换成int
purchase_list2['recency'] = purchase_list2['recency'].astype(int)

In [132]:
purchase_list2.groupby('clientId', as_index=True)['orderdate']

In [133]:
purchase_list2[purchase_list2.clientId == 1]['orderdate']

In [134]:
purchase_list2[purchase_list2.clientId == 1]['orderdate'].diff()

In [135]:
purchase_list2['interval'] = purchase_list2.groupby("clientId", #分类条件
                                  as_index = True # 分类条件是否要取代Index
                                  )['orderdate'].diff()

In [136]:
purchase_list2

In [137]:
purchase_list2.dropna(inplace=True)

In [138]:
purchase_list2.info()

In [139]:
purchase_list2['interval'] = purchase_list2['interval'].dt.days

In [140]:

# purchase_list2.dropna(inplace = True)#刪除第一次來本店的数据
# purchase_list2['interval'] = purchase_list2['interval'].astype(str) #转换成字符串
# purchase_list2['interval'] = purchase_list2['interval'].str.replace('days.*', '').astype(int) #删掉day字段

In [141]:
purchase_list2

In [142]:
purchase_list2

In [143]:
purchase_list2['cumsum'] = 1
purchase_list2['cumsum'] = purchase_list2.groupby("clientId")['cumsum'].cumsum()
purchase_list2

In [144]:
interval_mean = purchase_list2.groupby('clientId', as_index=False)['interval'].mean()

In [145]:
interval_mean.rename(columns={"interval": "interval_mean"}, inplace = True)

In [146]:
interval_mean

In [147]:
#算平均
interval_mean = purchase_list2.groupby("clientId", as_index = False)['interval'].mean()
interval_mean.rename(columns={"interval": "interval_mean"}, inplace = True)

In [148]:
# 合并平均
purchase_list2 =purchase_list2.merge(interval_mean, # 要合并的资料
                                   on = ['clientId'] # 两张表链接的Key
                                   ,how='inner') # 合并的方式

In [149]:
purchase_list2

In [150]:
purchase_list2['weighted_average'] = purchase_list2['interval'] * purchase_list2['cumsum'] / (purchase_list2['frequency']*(purchase_list2['frequency'] -1)/2) / purchase_list2['interval_mean']
purchase_list2

In [151]:
clientId_weighted_average = purchase_list2.groupby("clientId", as_index = False)['weighted_average'].sum()
clientId_weighted_average['weighted_average'] = 1-clientId_weighted_average['weighted_average']
clientId_weighted_average

In [153]:
clientId_weighted_average['back_probability'] = np.where( clientId_weighted_average['weighted_average'] >= 0, 'good','bad')

In [154]:
clientId_weighted_average

In [155]:
clientId_weighted_average['percentage'] = (clientId_weighted_average['weighted_average']  - clientId_weighted_average['weighted_average'].min()) / (clientId_weighted_average['weighted_average'].max() - clientId_weighted_average['weighted_average'].min()) *100

In [156]:
clientId_weighted_average

## 商品推荐清单

In [157]:
recommended_list = purchase_list2.groupby("clientId", #分类条件
                                  as_index = False # 分类条件是否要取代Index
                                  )['banana', 'milk', 'water'].sum() 

In [164]:
recommended_list[['banana', 'milk', 'water']].iloc[0].sort_values(ascending=False).index.values

In [165]:
' > '.join(['water', 'milk', 'banana'])

In [166]:
sort=[]
for i in range(len(recommended_list)):
    sort.append(' > '.join(recommended_list[['banana', 'milk', 'water']].iloc[i].sort_values(ascending=False).index.values))

In [168]:
clientId_weighted_average['recommended_list']=sort

In [169]:
clientId_weighted_average

In [170]:
#回来算Recency、Frequency
clientId_weighted_average['recency'] = purchase_list.groupby("clientId", #分类条件
                                                              as_index = False # 分类条件是否要取代Index
                                                              )['recency'].min()[['recency']] 
clientId_weighted_average['frequency'] = purchase_list.groupby("clientId", #分类条件
                                                              as_index = False # 分类条件是否要取代Index
                                                              )['frequency'].min()['frequency'] # 
clientId_weighted_average